# Experiment: SGR Quality Demo

Цель: показать baseline vs SGR, full LLM debug и DIF метрик из SQLite.


In [ ]:
from pathlib import Path
import sqlite3
import sys

sys.path.append('src')

db_path = Path('dialogs.db')
assert db_path.exists(), 'Run demo first: PYTHONPATH=src python3 -m dialogs.demo'
conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row


## 1) Базовая статистика хранилища


In [ ]:
tables = [
    'conversations','messages','rules','rule_versions','rule_results',
    'llm_runs','llm_calls','experiment_runs','experiment_metrics'
]
for t in tables:
    c = conn.execute(f'SELECT COUNT(*) FROM {t}').fetchone()[0]
    print(f'{t}: {c}')


## 2) Последние baseline/sgr run


In [ ]:
runs = conn.execute("""
SELECT run_id, mode, git_branch, git_commit, prompt_version, sgr_version, started_at_utc
FROM experiment_runs
ORDER BY started_at_utc DESC
LIMIT 10
""").fetchall()
for r in runs:
    print(dict(r))


## 3) Accuracy метрики по правилам


In [ ]:
rows = conn.execute("""
SELECT run_id, rule_key, metric_name, metric_value
FROM experiment_metrics
WHERE metric_name='accuracy'
ORDER BY rule_key, run_id
""").fetchall()
for r in rows[:50]:
    print(dict(r))


## 4) Full LLM debug: примеры вызовов


In [ ]:
calls = conn.execute("""
SELECT call_id, run_id, phase, attempt, parse_ok, validation_ok, error_message, request_json, response_json
FROM llm_calls
ORDER BY call_id DESC
LIMIT 5
""").fetchall()
for c in calls:
    print('---')
    print('meta', {k:c[k] for k in ['call_id','run_id','phase','attempt','parse_ok','validation_ok','error_message']})
    print('request_json', c['request_json'][:220], '...')
    print('response_json', c['response_json'][:220], '...')


## 5) Визуальный DIF accuracy

Артефакт хранится в `artifacts/accuracy_diff.png`.


In [ ]:
from IPython.display import Image, display
png = Path('artifacts/accuracy_diff.png')
if png.exists():
    display(Image(filename=str(png)))
else:
    print('No artifact found. Run: PYTHONPATH=src python3 -m dialogs.demo')
